# Chatbot Evaluation

## Import Packages

In [23]:
from langchain.prompts import ChatPromptTemplate
from langchain.memory.chat_message_histories import ChatMessageHistory

from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

## 6. Chatbot 만들기

### 기본 Chatbot


In [2]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 친절한 AI 비서야."),
    ("human", "{question}"),
])

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7
)

chain = prompt | llm

response = chain.invoke({"question": "오늘 날씨 어때?"})
print(response.content)

죄송해요, 저는 실시간으로 정보를 제공할 수 없어요. 하지만 날씨 앱이나 인터넷을 통해 확인하시면 될 거에요. 혹시 다른 도움이 필요하신가요?


### 메모리 기반 Chatbot

- ChatMessageHistory: 메세지들을 순차적으로 저장하는 클래스

In [19]:
# 세션별 메세지 히스토리 저장소
session_history = {}

def get_message_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in session_history:
        session_history[session_id] = ChatMessageHistory()
    return session_history[session_id]

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "친절한 AI 챗봇입니다."),
    ("placeholder", "{chat_history}"),
    ("human", "{question}")
])

chain = prompt | llm

chatbot = RunnableWithMessageHistory(
    chain,
    get_message_history, # 세선 별 메모리 관리자
    input_messages_key="question",
    history_messages_key="chat_history"
)

response 내부로직
1. 세션의 메세지 히스토리 가져오기
   - history = get_message_history("user_123")
2. LLM에게 전달될 최종 프롬프트 완성
   - formatted_prompt = {"chat_history: history.messages (이전 대화 내용), "question": question (현재 질문)}
3. LLM 응답 생성
   - response = llm.invoke(formatted_prompt)
4. 새로운 대화 내용 저장
   - history.add_user_message(question)
   - history.add_ai_message(reponse.content)

In [22]:
# 챗봇 사용
response = chatbot.invoke(
    {"question": "안녕하세요! 저는 29살입니다."}, 
    {"session_id": "user_123"}
)
print(response.content)

# 다음 대화 - 이전 대화 기록 유지
response = chatbot.invoke(
    {"question": "저는 몇살일까요?"}, 
    {"session_id": "user_123"}
)
print(response.content)

안녕하세요! 만나서 반가워요. 29살이라면 젊은 나이에요! 무엇을 도와드릴까요? 😊
죄송해요, 제가 사용자의 나이를 알 수 있는 방법은 없어요. 사용자분이 29살이라고 말씀해주셨으니, 그대로 받아들이겠어요. 혹시 다른 궁금한 점이 있으시면 언제든지 물어보세요! 😊
